In [23]:
from get_variable_names import *

In [24]:
from grids import *
import pandas as pd

In [25]:
bret = pd.read_excel("/home/enf/b2ar_analysis/bret_bias_study2.xlsx", header=0).set_index("EvanName")
bret = bret.iloc[[i for i in range(0,bret.shape[0]) if np.nan_to_num(bret["CID"].values[i]) != 0]]
bret.index = ["CID_%d" %cid for cid in bret["CID"].values]

In [40]:
base_docking_dir = "/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/clusterer_25clusters_1samples_samples_kdtree"
if not os.path.exists(base_docking_dir):
    os.makedirs(base_docking_dir)

precision = "SP"
docking_dir = "%s/docking_%s" %(base_docking_dir, precision)
if not os.path.exists(docking_dir):
    os.makedirs(docking_dir)

b2ar_ligand_dir = "/home/enf/b2ar_analysis/all_ligands/32-stereoisomers_6-ring-conf"

In [27]:
docking_dir

'/home/enf/md_simulations/B1AR/sparse-tICA_t50_n_components2contact_6pt6_angstroms-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/clusterer_25clusters_1samples_samples_kdtree/docking_SP'

In [29]:
mehrdad = pd.read_excel("/home/enf/md_simulations/B1AR/docking/mehrdad1.xlsx", header=1)
print(mehrdad.columns.values.tolist())
actions = []

classes = []
labels = mehrdad["Label "].values.tolist()
for i in range(0, mehrdad.shape[0]):
    if "unbiased" in labels[i].lower():
        actions.append("unbiased agonist")
        classes.append(0.)
    elif "biased" in labels[i].lower():
        actions.append("biased agonist")
        classes.append(1.)
    elif "non-binder" in labels[i].lower():
        actions.append("non-binder")
        classes.append(3.)
    elif "antagonist" in labels[i].lower():
        actions.append("antagonist")
        classes.append(3.)
    else:
        actions.append("unknown")
        classes.append(2.)
mehrdad["action"] = actions
mehrdad["class"] = classes

        
#bret = bret.iloc[[i for i in range(0,bret.shape[0]) if np.nan_to_num(bret["CID"].values[i]) != 0]]
#bret.index = ["CID_%d" %cid for cid in bret["CID"].values]



['Name ', 'Label ', 'Emax on G-Protein', 'Emax on B-arrestin', 'Reference ']


In [67]:
import grids
from importlib import reload
reload(grids)
from grids import *

b1ar_ligands_dir = "/home/enf/md_simulations/B1AR/docking/ligands/32-stereoisomers_6-ring-conf"

mehrdad.index = [n.lower().replace("-", "").replace(" ", "").strip() for n in mehrdad["Name "].values]
mehrdad_agonists = mehrdad.loc[mehrdad["class"] != 3]
mehrdad_agonists = [n for n in mehrdad_agonists.index.values.tolist() if "ks" in n or "std" in n]

bret.index = [n.lower().replace("-", "").replace(" ", "").strip() for n in bret.index]
bret_antagonists = bret.loc[bret["B1AR-Gprotein, Mean"] < 0.2].index.values.tolist()
bret_agonists = bret.loc[bret["B1AR-Gprotein, Mean"] > 0.2].index.values.tolist()
b1ar_agonists = bret_agonists + mehrdad_agonists
b1ar_agonism = pd.DataFrame([1. for a in b1ar_agonists] + [0. for b in bret_antagonists], columns=["agonism"], index=b1ar_agonists + bret_antagonists)

smiles_strings = convert_ligand_files_to_smiles(b1ar_ligands_dir, b1ar_agonism.index.values.tolist())
b1ar_agonism["smiles"] = smiles_strings

with open("/home/enf/htbc/b1ar_agonism_df.pkl", "wb") as f:
    pickle.dump(b1ar_agonism, f, protocol=2)

In [41]:
bret_antagonists = bret.loc[bret["B2AR-Gprotein, Mean"] < 0.2]
bret_agonists = bret.loc[bret["B2AR-Gprotein, Mean"] > 0.2]
b2ar_agonism = pd.DataFrame([1. for a in bret_agonists.index] + [0. for b in bret_antagonists.index], columns=["agonism"], index=bret_agonists.index.tolist() + bret_antagonists.index.tolist())

smiles_strings = convert_ligand_files_to_smiles(b2ar_ligand_dir, b2ar_agonism.index.values.tolist())
b2ar_agonism["smiles"] = smiles_strings

with open("/home/enf/htbc/b2ar_agonism_df.pkl", "wb") as f:
    pickle.dump(b2ar_agonism, f, protocol=2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


In [27]:
mehrdad

,Name,Label,Emax on G-Protein,Emax on B-arrestin,Reference,action,class
isoproterenol,Isoproterenol,Full agonist/unbiased,100,100,NaN,unbiased agonist,0.0
adrenalone,Adrenalone,Full agonist/unbiased,87,87,"Casella et al., 2011",unbiased agonist,0.0
epinephrine,Epinephrine,Full agonist/unbiased,100,100,"Casella et al., 2011",unbiased agonist,0.0
fenoterol,Fenoterol,Full agonist/unbiased,81,87,"Casella et al., 2011",unbiased agonist,0.0
hexoprenaline,Hexoprenaline,Full agonist/unbiased,101,96,"Casella et al., 2011",unbiased agonist,0.0
norepinephrine,Norepinephrine,Full agonist/unbiased,100,95,"Casella et al., 2011",unbiased agonist,0.0
ksn01014,KSN01-01-4,Full agonist for G-protein/not tested in beta-...,99.95,NaN,NaN,unknown,2.0
ksn01017,KSN01-01-7,Full agonist for G-protein/not tested in beta-...,94.7,NaN,NaN,unknown,2.0
ksn01018,KSN01-01-8,Full agonist for G-protein/not tested in beta-...,107.1,NaN,NaN,unknown,2.0
ksn010114,KSN01-01-14,Full agonist for G-protein/biased for G-protein,95.7,No response,NaN,biased agonist,1.0


In [6]:
import pubchempy as pcp
mehrdad.index = mehrdad["Name "]
new_index = []
for n in mehrdad["Name "].values.tolist():
    cids = pcp.get_cids(n, 'name', 'substance', list_return='flat')
    cids = ["cid_%s" %cid for cid in cids]
    replace=False
    for cid in cids:
        if cid in full_docking_df.index.values:
            replace=True
            new_index.append(cid)
            print(n)
            print(cid)
            break
    if not replace:
        new_index.append(n)
        
        

NameError: name 'full_docking_df' is not defined

In [226]:
full_ligand_df = pd.concat([bret_y_df.loc[bret_y_df["class"] >= 0.], mehrdad], axis=0)["class"]#.loc[mehrdad["class"] != 4.], mehrdad.loc[mehrdad["class"] == 3.]], axis=0)["class"]#.loc[[n for n in mehrdad.index.values if "ks" in n.lower() or mehrdad["class"].loc[n] == 0. and mehrdad["class"].loc[n] != 4.]]], axis=0)["class"]
full_ligand_df = full_ligand_df.reset_index().drop_duplicates(subset='index', keep='first').set_index('index')
other_inactives = [1978, 59768, 4171, 9966051]
for cid in ["CID_%d" %i for i in other_inactives]:
    full_ligand_df.loc[cid] = [0.]

#full_ligand_df = full_ligand_df.loc[full_ligand_df["class"] != 2.]

In [227]:
ligand_dir = b1ar_lig_dir
ligand_files = get_trajectory_files(ligand_dir, ".sdf") + get_trajectory_files(ligand_dir, ".mol")
ligand_files_clean = [n.split(".")[0].split("/")[-1].lower().strip() for n in ligand_files]

full_ligand_df["smiles"] = ""

removed_ligands = []
sdf_files = []
sdf_ligands = []

for ligand in full_ligand_df.index:
    print(ligand)
    if (len(str(full_ligand_df["smiles"].loc[ligand])) > 4):
        continue
    else:
        try:
            sdf_file = ligand_files[ligand_files_clean.index(ligand.lower())]
            sdf_files.append(sdf_file)
            sdf_ligands.append(ligand)
        except:
            removed_ligands.append(ligand)

smiles_list = convert_sdfs_to_smiles(sdf_files, parallel=True, worker_pool=None)
full_ligand_df["smiles"].loc[sdf_ligands] = smiles_list
#full_ligand_df = full_ligand_df.loc[[n for n in full_ligand_df.index.values if n not in removed_ligands]]

full_ligand_df

CID_7436
CID_10443654
CID_161394
CID_2083
CID_2119
CID_2249
CID_2405
CID_18026
CID_5606
CID_2585
CID_2687
CID_2755
CID_2783
CID_5806
CID_36811
CID_681
CID_10353067
CID_10087493
CID_838
CID_3343
CID_3609
CID_3682
CID_4546390
CID_123686
CID_23843
CID_3779
CID_3869
CID_913
CID_217246
CID_4382
CID_951
CID_1237
CID_4086
CID_4828
CID_4883
CID_4930
CID_4946
CID_33572
CID_5241141
CID_5253
CID_170373
CID_5403
CID_23844
CID_5478
CID_155774
cid_3019
cid_7436
cid_838
cid_3343
cid_3609
cid_951
ksn01014
ksn01017
ksn01018
ksn010114
ksn010115
ksn010126
ks01502
ks01504
ks01509
ks01515
ks01501
ks01503
ks01506
ks01507
ks01510
ks01511
ksn010118
std101c4
cid_313
cid_2405
cid_3682
cid_2685
cid_2369
cid_2249
cid_18026
cid_3869
cid_4883
cid_4930
cid_4946
cid_5478
cid_3152
cid_5732
cid_3100
cid_3689
cid_3162
cid_4614
cid_4848
cid_71771
cid_1981
cid_2554
sulfadizaine
cid_4539
cid_47964
CID_1978
CID_59768
CID_4171
CID_9966051


,class,smiles
index,,
CID_7436,1.0,CNCC(=O)c1ccc(c(c1)O)O\t7436\n
CID_10443654,1.0,OCc1cc(ccc1O)C(CNC(C)C)O\t10443654\n
CID_161394,0.0,OC(c1ccc(c(c1)C(=O)N)O)CNC(C)(C)C\t161394\n
CID_2083,1.0,OCc1cc(ccc1O)C(CNC(C)(C)C)O\t2083\n
CID_2119,1.0,C=CCc1ccccc1OCC(CNC(C)C)O\t2119\n
CID_2249,0.0,OC(COc1ccc(cc1)CC(=O)N)CNC(C)C\t2249\n
CID_2405,0.0,OC(COc1ccc(cc1)COCCOC(C)C)CNC(C)C\t2405\n
CID_18026,0.0,COc1ccc(cc1C(C(NC(C)(C)C)C)O)OC\t18026\n
CID_5606,1.0,OC(c1ccccc1Cl)CNC(C)(C)C\t5606\n


In [50]:
b1ar_lig_dir = "/home/enf/md_simulations/B1AR/docking/ligands/32-stereoisomers_6-ring-conf"
f#download_sdfs_from_cids(other_inactives, b1ar_lig_dir, worker_pool=None, parallel=True)

In [42]:
import pandas as pd
chembl_df = pd.read_excel("/home/enf/md_simulations/B1AR/docking/bioactivity-17_7_12_38.xlsx", header=0, index_col=0)#.set_index("CMPD_CHEMBLID")
#chembl_df = pd.read_excel("/home/enf/md_simulations/MOR/docking/chembl_ec50.xlsx", header=0, index_col=0)#.set_index("CMPD_CHEMBLID")

print(chembl_df.shape)
chembl_df["smiles"] = chembl_df["CANONICAL_SMILES"]
chembl_df.index = [n.lower() for n in chembl_df.index]
chembl_df = chembl_df.groupby(chembl_df.index).first()

chembl_df["ligand"] = chembl_df.index

inactive_cutoff = 7.
active_cutoff = 7.

chembl_inactives = list(set(chembl_df.loc[chembl_df["ACTIVITY_COMMENT"].fillna('').str.contains("Not Active")].ligand.values.tolist() + chembl_df.loc[chembl_df["PCHEMBL_VALUE"] < inactive_cutoff].ligand.values.tolist()))
chembl_actives = chembl_df.loc[chembl_df["PCHEMBL_VALUE"] > active_cutoff].ligand.values.tolist()
intersection = set(chembl_inactives).intersection(set(chembl_actives))
chembl_inactives = list(set(chembl_inactives).difference(intersection))
chembl_actives = list(set(chembl_actives).difference(intersection)) 

ligand_df = pd.DataFrame([(0., "dud") for a in chembl_inactives] + [(1., "binder") for a in chembl_actives], columns=["class", "activity"], index = chembl_inactives + chembl_actives)
ligand_df["smiles"] = chembl_df.loc[ligand_df.index.values.tolist()]["smiles"]



print("n inactives: %d" %len(chembl_inactives))
print("n actives: %d" %len(chembl_actives))

(1254, 58)
n inactives: 1059
n actives: 94


In [71]:
other_b1ar_actives = pd.DataFrame([[1., "binder", smi] for smi in b1ar_agonism["smiles"].values], columns=["class", "activity", "smiles"], index=b1ar_agonism.index)
ligand_df = pd.concat([ligand_df, other_b1ar_actives], axis=0)

In [73]:
#INCLUDE KS AND BRET LIGANDS FOR AFFINITY AS WELL.

In [46]:
"""full_ligand_df = chembl_df.copy()

print(full_ligand_df.shape)
c_maxes = full_ligand_df.groupby(full_ligand_df.index).PCHEMBL_VALUE.transform(max)
full_ligand_df = full_ligand_df[full_ligand_df.PCHEMBL_VALUE == c_maxes]
print(full_ligand_df.shape)
full_ligand_df = full_ligand_df.groupby(full_ligand_df.index).first()
print(full_ligand_df.shape)
print(full_ligand_df.loc[full_ligand_df["PCHEMBL_VALUE"] < 7.].shape)

full_ligand_df = full_ligand_df.loc[full_ligand_df["PCHEMBL_VALUE"].dropna().index]

full_ligand_df.index = full_ligand_df.ligand
full_ligand_df.smiles = full_ligand_df.CANONICAL_SMILES
full_ligand_df = full_ligand_df.loc[full_ligand_df.smiles.dropna().index.values.tolist()]
full_ligand_df = full_ligand_df.groupby(full_ligand_df.index).first()


inactive_cutoff = 7.
active_cutoff = 7.

chembl_inactives = list(set(chembl_df.loc[chembl_df["ACTIVITY_COMMENT"].fillna('').str.contains("Not Active")].ligand.values.tolist() + full_ligand_df.loc[full_ligand_df["PCHEMBL_VALUE"] < inactive_cutoff].ligand.values.tolist()))
chembl_actives = full_ligand_df.loc[full_ligand_df["PCHEMBL_VALUE"] > active_cutoff].ligand.values.tolist()
intersection = set(chembl_inactives).intersection(set(chembl_actives))
chembl_inactives = list(set(chembl_inactives).difference(intersection))
chembl_actives = list(set(chembl_actives).difference(intersection))
print("final n inactives: %d" %len(chembl_inactives))
print("final n actives: %d" %len(chembl_actives))

full_ligand_df = full_ligand_df.loc[chembl_inactives + chembl_actives]

full_ligand_df["class"] = [0. for i in chembl_inactives] + [1. for j in chembl_actives]
full_ligand_df["activity"] = ["dud" for i in chembl_inactives] + ["binder" for j in chembl_actives]

#print(full_ligand_df.shape)
#print(len(chembl_inactives + chembl_actives))
full_ligand_df.index = [str(n).lower().strip().replace(" ", "_").replace("β", "beta").replace("α", "alpha").replace("Α", "alpha") for n in full_ligand_df.index]
#full_ligand_df.shape
"""

(1254, 60)
(313, 60)
(307, 60)
(209, 60)
final n inactives: 1058
final n actives: 95


In [47]:
full_ligand_df["activity"]

chembl319244        dud
chembl316561        dud
chembl3290985       dud
chembl1909064       dud
chembl1200656       dud
chembl1200396       dud
chembl608           dud
chembl557555        dud
chembl1651906       dud
chembl502620        dud
chembl1255834       dud
chembl1055          dud
chembl514           dud
chembl3290975       dud
chembl439           dud
chembl1095699       dud
chembl196           dud
chembl26            dud
chembl1504          dud
chembl783           dud
chembl1909055       dud
chembl311350        dud
chembl427216        dud
chembl54440         dud
chembl1650          dud
chembl1460          dud
chembl306043        dud
chembl61            dud
chembl268291        dud
chembl62193         dud
chembl844           dud
chembl460499        dud
chembl19299         dud
chembl41062         dud
chembl1258597       dud
chembl184           dud
chembl3704833       dud
chembl1221540       dud
chembl21            dud
chembl427711        dud
chembl460291        dud
chembl444633    

In [74]:
with open("/home/enf/htbc/b1ar_affinity_with_bret_mehrdad_df.pkl", "wb") as f:
    pickle.dump(ligand_df, f, protocol=2)

In [ ]:
from efficacy_scripts import *
from grids import *

with open("/home/enf/b2ar_analysis/all_ligands/chembl_df.pkl", "rb") as f:
    chembl_df = pickle.load(f)
inactive_cutoff = 7.
active_cutoff = 7.

print(chembl_df.shape)
chembl_df["smiles"] = chembl_df["CANONICAL_SMILES"]
chembl_df.index = [n.lower() for n in chembl_df.index]
chembl_df = chembl_df.groupby(chembl_df.index).first()

chembl_df["ligand"] = chembl_df.index
compounds = convert_smiles_to_compounds(chembl_df["smiles"].values.tolist(), parallel=False, worker_pool=dview)
chembl_df["CID"] = [c[3] for c in compounds]
chembl_df["ligand"] = chembl_df.index
chembl_df["ligand"].loc[chembl_df["CID"] != ""] = ["CID_%d" %cid for cid in chembl_df.loc[chembl_df["CID"] != ""]["CID"].values.tolist()]


inactive_cutoff = 7.
active_cutoff = 7.

chembl_inactives = list(set(chembl_df.loc[chembl_df["ACTIVITY_COMMENT"].fillna('').str.contains("Not Active")].ligand.values.tolist() + chembl_df.loc[chembl_df["PCHEMBL_VALUE"] < inactive_cutoff].ligand.values.tolist()))
chembl_actives = chembl_df.loc[chembl_df["PCHEMBL_VALUE"] > active_cutoff].ligand.values.tolist()
intersection = set(chembl_inactives).intersection(set(chembl_actives))
chembl_inactives = list(set(chembl_inactives).difference(intersection))
chembl_actives = list(set(chembl_actives).difference(intersection)) 

ligand_df = pd.DataFrame([(0., "dud") for a in chembl_inactives] + [(1., "binder") for a in chembl_actives], columns=["class", "activity"], index = chembl_inactives + chembl_actives)
ligand_df["smiles"] = chembl_df.loc[ligand_df.index.values.tolist()]["smiles"]
print("n inactives: %d" %len(chembl_inactives))
print("n actives: %d" %len(chembl_actives))

(1453, 61)
n inactives: 1030
n actives: 210


/home/enf/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
ligand_df = pd.DataFrame([(0., "dud") for a in chembl_inactives] + [(1., "binder") for a in chembl_actives], columns=["class", "activity"], index = chembl_inactives + chembl_actives)
chembl_df["old_index"] = chembl_df.index
chembl_df.index = chembl_df.ligand
ligand_df["smiles"] = chembl_df.loc[ligand_df.index.values.tolist()]["smiles"]
print("n inactives: %d" %len(chembl_inactives))
print("n actives: %d" %len(chembl_actives))

n inactives: 1030
n actives: 210


In [20]:
ligand_df["smiles"]

CID_45006123                              [Cl-].C[n+]1ccccc1\C=N\O
CID_10287658        COc1ccc(Nc2ccc(CCNCC(O)c3ccc(O)c(CO)c3)cc2)cc1
CID_10219        CC[C@H]1CN2CCc3cc(OC)c(OC)cc3[C@@H]2C[C@@H]1C[...
CID_6279         C[C@H]1C[C@@H]2[C@H](CC[C@@]3(C)[C@H]2CC[C@]3(...
CID_57363        CC(C)(C)NC(=O)[C@H]1CC[C@H]2[C@@H]3CC[C@H]4NC(...
CID_2244                                     CC(=O)Oc1ccccc1C(=O)O
CID_1548953      CCN(CC)CCOc1ccc(cc1)\C(=C(\Cl)/c2ccccc2)\c3ccccc3
CID_9992825                       CC(C)NC1CCN2C(=O)Nc3cccc(C1O)c23
CID_5282230                COc1ccc(\C=C\C(=O)Nc2ccccc2C(=O)O)cc1OC
CID_169371       Nc1nc(N)c2nc(CNc3ccc(cc3)C(=O)N[C@@H](CCC(=O)O...
CID_5694                       Cc1cccc(Nc2cc(Cl)nc(SCC(=O)O)n2)c1C
CID_2162           CCOC(=O)C1=C(COCCN)NC(=C(C1c2ccccc2Cl)C(=O)OC)C
CID_2435                               Brc1c(NC2=NCCN2)ccc3nccnc13
CID_4495                CS(=O)(=O)Nc1ccc(cc1Oc2ccccc2)[N+](=O)[O-]
CID_16736514                    O[C@H](CNCCc1ccccc1)c2cc(O)cc(

In [15]:
[n for n in chembl_df.ligand if "CID" in n]

['CID_176155',
 'CID_123600',
 'CID_2141',
 'CID_6047',
 'CID_4781',
 'CID_1367',
 'CID_2541',
 'CID_5509',
 'CID_3690',
 'CID_5994',
 'CID_2812',
 'CID_5280795',
 'CID_564',
 'CID_10130293',
 'CID_5746',
 'CID_5560',
 'CID_2732',
 'CID_3365',
 'CID_6238',
 'CID_54454',
 'CID_9681',
 'CID_34312',
 'CID_60846',
 'CID_6167',
 'CID_4409',
 'CID_4614',
 'CID_3478',
 'CID_5511',
 'CID_60726',
 'CID_2554',
 'CID_33613',
 'CID_46889811',
 'CID_46889812',
 'CID_46889852',
 'CID_46889853',
 'CID_5187268',
 'CID_46889809',
 'CID_46889699',
 'CID_46889727',
 'CID_46889643',
 'CID_46889728',
 'CID_2769774',
 'CID_4080436',
 'CID_46889769',
 'CID_46889770',
 'CID_2770390',
 'CID_3687256',
 'CID_2796082',
 'CID_24321330',
 'CID_4143663',
 'CID_4276762',
 'CID_46889642',
 'CID_46889771',
 'CID_46889772',
 'CID_46889670',
 'CID_3654892',
 'CID_46889767',
 'CID_46889768',
 'CID_46889700',
 'CID_46889701',
 'CID_46889729',
 'CID_46889730',
 'CID_4133',
 'CID_4421983',
 'CID_46889854',
 'CID_46889810',
 

In [ ]:
chembl_df.index

In [21]:
with open("/home/enf/htbc/b2ar_affinity_df_cid_convert.pkl", "wb") as f:
    pickle.dump(ligand_df, f, protocol=2)

In [ ]:
import pandas as pd
chembl_df = pd.read_excel("/home/enf/md_simulations/b2ar_analysis/", header=0, index_col=0)#.set_index("CMPD_CHEMBLID")
#chembl_df = pd.read_excel("/home/enf/md_simulations/MOR/docking/chembl_ec50.xlsx", header=0, index_col=0)#.set_index("CMPD_CHEMBLID")

print(chembl_df.shape)
chembl_df["smiles"] = chembl_df["CANONICAL_SMILES"]
chembl_df.index = [n.lower() for n in chembl_df.index]
chembl_df["ligand"] = chembl_df.index
#compounds = convert_smiles_to_compounds(chembl_df["smiles"].values.tolist(), parallel=False, worker_pool=dview)
#chembl_df["CID"] = [c[3] for c in compounds]
#chembl_df["ligand"] = chembl_df.index
#chembl_df["ligand"].loc[chembl_df["CID"] != ""] = ["CID_%d" %cid for cid in chembl_df.loc[chembl_df["CID"] != ""]["CID"].values.tolist()]

#write_smiles_files(chembl_df, "/home/enf/md_simulations/B1AR/docking/ligands/32-stereoisomers_6-ring-conf")


#undone_ligands = list(set(chembl_df.loc[chembl_df["CID"] == ""].index.values.tolist() +  [n for n in chembl_cid_names if n not in get_ligands("/home/enf/b2ar_analysis/all_ligands/32-stereoisomers_6-ring-conf", ".sdf")]))

#print(len(chembl_cids))
#print(len(undone_ligands))

In [8]:
import os
from ipyparallel import Client
rc = Client()
print(len(rc.ids))
dview = rc[:]
dview.map(os.chdir, ['/home/enf/b2ar_analysis/conformation']*len(rc.ids))

110


<AsyncMapResult: chdir>

In [35]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)